In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
embeddings = None

[nltk_data] Downloading package punkt to /home/jorge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jorge/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jorge/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Spliteo para obtener vectores de train y test

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train,test = train_test_split(train_df,test_size=0.33,random_state = 17)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
y_train = train['target'].values
y_test = test['target'].values

In [5]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [6]:
processed, X_train = prepare_for_cnn(train)

5100
Percentage of words covered in the embeddings = 0.5339805825242718


In [7]:
processed

,index,text_embedding_0,text_embedding_1,text_embedding_2,text_embedding_3,text_embedding_4,text_embedding_5,text_embedding_6,text_embedding_7,text_embedding_8,...,text_embedding_290,text_embedding_291,text_embedding_292,text_embedding_293,text_embedding_294,text_embedding_295,text_embedding_296,text_embedding_297,text_embedding_298,text_embedding_299
0,2247,0.174561,-0.187804,-0.174774,0.391602,-0.800507,1.263681,-0.333313,-0.530518,1.439468,...,-3.093704,-0.026474,-1.963478,0.668640,0.470688,0.261047,0.084839,-0.743698,1.064865,-1.014771
1,6872,-0.832794,0.522888,1.680176,2.143860,-0.971680,-0.209106,0.139069,-3.171082,2.443481,...,-1.377075,-0.090576,-1.615211,0.426344,-1.479126,-0.139343,-1.033325,-1.442383,1.079529,-0.524834
2,82,0.658539,-0.151917,1.475098,1.222046,-0.955200,0.176971,0.430298,-1.353149,1.527988,...,-0.303158,0.671021,-1.591064,0.691040,-0.726196,-0.286438,-0.898315,-0.965561,0.320984,-0.436462
3,1997,1.228882,0.942330,0.987427,0.798660,-1.113495,1.210213,0.925262,-0.703918,0.535461,...,-0.910240,0.496979,-1.143379,0.590332,0.169312,-0.548679,-0.295044,-1.006866,-0.003662,-0.747009
4,5227,0.322754,0.200623,0.307800,0.546387,-0.006531,0.135468,-0.101074,-1.114014,1.345947,...,0.023804,-0.064453,-0.679199,0.453125,-0.692871,-0.314453,-0.340820,-0.665649,-0.068527,-0.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095,406,0.113220,0.679993,1.102051,0.953125,-0.223755,0.406830,0.233826,-1.125244,1.049072,...,0.369751,-1.126282,-0.685760,1.176270,-0.364258,-0.521740,-0.972382,-1.240234,-0.127380,1.053711
5096,5510,-1.314148,0.022621,0.202881,1.220764,-1.881714,0.512215,-0.020142,-0.086838,2.091187,...,-0.970459,1.073730,0.985413,1.380371,-0.703445,-0.206177,0.860336,-0.740601,-0.217972,1.305298
5097,2191,-0.438354,-0.308868,-0.638916,0.773682,-0.996460,-0.459381,-0.472260,-2.293091,1.772034,...,-0.197659,0.354160,-0.334763,0.917114,0.396240,-0.645752,-1.683411,-0.902588,0.569000,0.130371
5098,7409,1.274780,1.120117,2.076721,-0.992424,1.082748,-0.778595,0.857422,-2.068481,0.129395,...,1.093018,-0.877655,-0.676270,0.183716,-0.272217,1.118835,-1.630981,-2.040894,1.220459,0.724030


In [8]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)])
    return model

In [9]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 128)               187392    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 195,713
Trainable params: 195,713
Non-trainable params: 0
_________________________________________________________________


In [11]:
#es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
history = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2)#, callbacks=[es])

Epoch 1/50
128/128 [==============================] - 1s 9ms/step - loss: 0.3705 - accuracy: 0.8319 - val_loss: 0.5131 - val_accuracy: 0.7931
Epoch 2/50
128/128 [==============================] - 1s 8ms/step - loss: 0.3667 - accuracy: 0.8336 - val_loss: 0.5014 - val_accuracy: 0.7922
Epoch 3/50
128/128 [==============================] - 1s 8ms/step - loss: 0.3731 - accuracy: 0.8289 - val_loss: 0.5099 - val_accuracy: 0.7922
Epoch 4/50
128/128 [==============================] - 1s 7ms/step - loss: 0.3604 - accuracy: 0.8363 - val_loss: 0.5255 - val_accuracy: 0.7902
Epoch 5/50
128/128 [==============================] - 1s 6ms/step - loss: 0.3620 - accuracy: 0.8395 - val_loss: 0.5257 - val_accuracy: 0.7873
Epoch 6/50
128/128 [==============================] - 1s 8ms/step - loss: 0.3635 - accuracy: 0.8348 - val_loss: 0.5091 - val_accuracy: 0.8000
Epoch 7/50
128/128 [==============================] - 1s 8ms/step - loss: 0.3625 - accuracy: 0.8368 - val_loss: 0.5074 - val_accuracy: 0.8020
Epoch 

In [15]:
processed, final_test = prepare_for_cnn(test)
prediction = model.predict_classes(final_test)

2513
Percentage of words covered in the embeddings = 0.6020887228130689


In [16]:
accuracy_score(test['target'], prediction)

0.8101870274572225

In [18]:
f1_score(test['target'], prediction)

0.7557603686635944